In [143]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import pymc3 as pm
import arviz as az

In [144]:
Data_PATH = '/net/meso/work/svivier/Git/Data/' #Path for output (and input for other Scripts) Data
Plot_PATH = '/net/meso/work/svivier/Git/Plots/' #Path for Output Plots

In [145]:
COPEPOD80_df_clean = pd.read_csv(Data_PATH + 'Abundance_IM_final.csv')

In [147]:
COPEPOD80_df_clean

,Acartia,Acartia bifilosa,Acartia longiremis,Acartia spp.,Aetideopsis multiserrata,Aglantha digitale,Aglantha spp.,Alteutha interrupta,Augaptilus glacialis,Balanus balanus,...,Thysanoessa inermis,Thysanoessa longicaudata,Thysanoessa raschii,Thysanoessa spp.,Tisbe furcata,Undinella oblonga,Undinopsis spp.,Verruca spp.,Zaus goodsiri,Zaus spp.
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1023,0.0,0.0,0.0,0.19,0.0,0.0,0.0,0.0,0.0,0.0,...,0.37,0.0,0.0,0.0,0.0,0.0,0.19,0.38,0.0,0.0
1024,0.0,0.0,0.0,16.74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.93,0.0,0.0,0.0,0.0,0.0,2.79,3.72,0.0,0.0
1025,0.0,0.0,0.0,16.12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.62,31.63,0.0,0.0


# 1% + We create the simulated data: Simulated Abundance data = COPEPOD80_df_clean; Simulated Total Biomass data: COPEPOD80_df_clean x ICWs AtlantECO
We simulate data using our abundance data and the ICWs of AtlantECO=> We can test our model

Here, All species that has a maximum abundance <1% in each samples are grouped in Size groups (=1; <1; <<1)

In [148]:
groupes = {
    '=1': ['Aetideopsis multiserrata', 'Chiridius obtusifrons', 'Gaetanus brevispinus', 'Paraheterorhabdus compactus', 'Eukrohnia hamata', 'Rathkea spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    '<1': [ 'Mimocalanus distinctocephalus','Pseudochirella spectabilis', 'Aglantha spp.', 'Candacia armata', 'Haloptilus acutifrons', 'Heterorhabdus norvegicus', 'Mormonilla minor', 'Spinocalanus magnus', 'Spinocalanus spinosus'], 
    '<<1': ['Alteutha interrupta', 'Cyclopina longicornis', 'Danielssenia typica', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Laophonte perplexa', 'Lubbockia glacialis', 'Parathalestris croni', 'Parathalestris harpactoides', 'Rhynchothalestris helgolandica', 'Tegastes falcatus'],
    'inconnu': ['Euphysa spp.','Syllides spp.','Rathkea octopunctata','Discoconchoecia elegans','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.','Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.']


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [149]:
#=Priors corrigée: /10 les priors des copepods

priors = {
    '=1': {'mu': 1, 'sigma': 1},
    '<1': {'mu': 0.1, 'sigma': 0.1},
    '<<1': {'mu': 0.01, 'sigma': 0.01},
    'inconnu': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 5},
    'Evadne': {'sigma': 5},
    'Verruca': {'sigma':1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma':1},
    'Beroe': {'mu': 7.12, 'sigma': 10},
    'Acartia': {'mu': 0.003, 'sigma': 0.1},
    'Metridia': {'mu': 0.08, 'sigma': 0.1},
    'Paracalanus': {'mu': 0.0016, 'sigma': 0.01},
    'Temora': {'mu': 0.001, 'sigma': 0.01},
    'Oithona': {'mu': 0.0005, 'sigma': 0.01},
    'Spinocalanus': {'mu': 0.2, 'sigma': 0.2},
    'Centropages': {'mu': 0.0075, 'sigma': 0.1},
    'Sagitta': {'mu': 0.6, 'sigma': 1}, #Chaetognatha
    'Pseudocalanus': {'mu': 0.004, 'sigma': 0.1},
    'Thysanoessa': {'mu': 6, 'sigma': 6}, #Euphosiacea
    'Oikopleura': {'mu': 0.006, 'sigma': 0.01},
    'Microcalanus': {'mu': 0.0002, 'sigma': 0.01},
    'Microsetella spp.': {'mu': 0.0008, 'sigma': 0.01},
    'Calanus finmarchicus': {'mu': 0.2, 'sigma': 1},
    'Calanus glacialis': {'mu': 0.2, 'sigma': 1},
    'Calanus hyperboreus': {'mu': 1.7, 'sigma': 1},
    'Bradyidius': {'mu': 0.3, 'sigma': 1},
    'Semibalanus': {'sigma': 5},
    'Chaetognatha': {'mu': 1, 'sigma': 1},
    'Euchaeta':  {'mu': 0.14, 'sigma': 1},
    'Eukrohnia': {'mu': 1.4, 'sigma': 1.4},
    'Hyperiidea':{'mu': 20, 'sigma': 20},
    'Themisto': {'mu': 1.4, 'sigma': 1.4},
    'Bolinopsis': {'mu': 40, 'sigma': 10},
    'Monstrilla': {'mu': 0.1, 'sigma': 1},
    'Oncaea': {'mu': 0.001, 'sigma': 0.1},
    'Pagurus':{'sigma': 5},
    'Paraeuchaeta': {'mu': 1, 'sigma': 1},
    'Pleurobrachia': {'mu': 2, 'sigma': 2},
    'Podon': {'sigma': 1},
    'Polychaeta':{'mu': 0.05, 'sigma': 0.05},
    'Thalestris':{'mu': 0.001, 'sigma': 0.01},
    'Tisbe':{'mu': 0.001, 'sigma': 0.01},
    'Undinella':{'mu': 0.3, 'sigma': 1},
    'Augaptilus':{'mu': 0.3, 'sigma': 1}, 
    'Obelia':{'mu': 1, 'sigma': 1},
    'Plotocnide':{'mu': 1, 'sigma': 1}, 
    'Scaphocalanus':{'mu': 0.2, 'sigma': 1},
    'Meganyctiphanes':{'mu': 10, 'sigma': 10}, 
    'Harpacticus':{'mu': 0.001, 'sigma': 0.01},
    'Calanus spp.':{'mu': 0.7, 'sigma': 0.7}, 
    'Gammaridea':{'mu': 3, 'sigma': 3}, 
    'Hormiphora cucumis':{'mu': 10, 'sigma': 10},
    'Hyas spp.':{'sigma': 1}, 
    'Jaschnovia tolli':{'mu': 3, 'sigma': 3}, 
    'Microsetella spp.':{'mu': 0.001, 'sigma': 0.01},
    'Scolecithricella minor':{'mu': 0.1, 'sigma': 0.1}, 
    'Spionidae spp.':{'sigma': 1}, 
    'Temorites brevis':{'mu': 0.3, 'sigma': 1}
    
    
    


    
    

}

In [150]:
# SpeciesCmass: the ICWs used to simulate the total biomass of the samples = ICWs of AtlantECO if available, 1 if not.

import pandas as pd

# Créer une liste pour stocker les tuples (espèce, valeur)
data = []

for species, values in priors.items():
    # Sélectionner 'mu' s'il est présent, sinon 'sigma'
    value = values.get('mu', values['sigma'])
    data.append((species, value))

# Créer un DataFrame à partir de la liste des tuples
SpeciesCmass = pd.DataFrame(data, columns=['Species', 'Value']).set_index('Species')

# Affichage du DataFrame
SpeciesCmass


,Value
Species,
=1,1.0000
<1,0.1000
<<1,0.0100
inconnu,1.0000
Limacina,1.0000
Undinopsis,1.0000
Balanus,5.0000
Evadne,5.0000
Verruca,1.0000


In [151]:
import pandas as pd


# Assurez-vous que les indices de SpeciesCmass correspondent aux colonnes de COPEPOD80_reduite
SpeciesCmass = SpeciesCmass.reindex(COPEPOD80_reduite.columns)

# Calculer la biomasse totale pour chaque échantillon
RealTotalCmass = (COPEPOD80_reduite * SpeciesCmass['Value']).sum(axis=1)




In [152]:
RealTotalCmass

0        10.450000
1       105.770000
2         5.900000
3        51.610000
4        43.200000
           ...    
1022      0.004277
1023      7.553807
1024     35.969606
1025     75.938797
1026     14.852089
Length: 1027, dtype: float64

In [153]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hyas spp., Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Semibalanus, Bradyidius, Calanus hyperboreus, Calanus glacialis, Calanus finmarchicus, Microsetella spp., Microcalanus, Oikopleura, Thysanoessa, Pseudocalanus, Sagitta, Centropages, Spinocalanus, Oithona, Temora, Paracalanus, Metridia, Acartia, Beroe, Copepoda, Fritillaria, Verruca, Evadne, Balanus, Undinopsis, Limacina, inconnu, <<1, <1, =1]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 270 seconds.


In [154]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df



Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
=1,1.283,0.002,2.629
<1,0.127,0.000,0.260
<<1,0.013,0.000,0.027
inconnu,0.807,0.000,1.917
Limacina,0.917,0.387,1.456
Undinopsis,0.802,0.001,1.862
Balanus,4.760,3.905,5.673
Evadne,4.841,3.092,6.464
Verruca,0.913,0.367,1.451


In [155]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Aetideopsis multiserrata,1.283,0.002,2.629
Chiridius obtusifrons,1.283,0.002,2.629
Gaetanus brevispinus,1.283,0.002,2.629
Paraheterorhabdus compactus,1.283,0.002,2.629
Eukrohnia hamata,1.283,0.002,2.629
...,...,...,...
Hyas spp.,0.799,0.001,1.877
Jaschnovia tolli,3.753,0.014,7.891
Scolecithricella minor,0.129,0.002,0.260


In [156]:
groupes = {
    'unknow': ['Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (group 1%)', 'HDI 3% (group 1%)', 'HDI 97% (group 1%)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (group 1%)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (group 1%)': data_subset['HDI 3%'].mean(),
            'HDI 97% (group 1%)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (group 1%)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (group 1%)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (group 1%)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
f1=grouped_results.copy()


In [157]:
f1 = pd.concat([f1, SpeciesCmass], axis=1)
f1

,Biomasse moyenne (group 1%),HDI 3% (group 1%),HDI 97% (group 1%),Value
unknow,0.8062,0.0001,1.913,NaN
Acartia,0.0730,0.0000,0.174,0.0030
Metridia,0.0910,0.0010,0.192,0.0800
Microcalanus,0.0080,0.0000,0.019,0.0002
Temora,0.0080,0.0000,0.020,0.0010
...,...,...,...,...
=1,NaN,NaN,NaN,1.0000
<1,NaN,NaN,NaN,0.1000
<<1,NaN,NaN,NaN,0.0100
inconnu,NaN,NaN,NaN,1.0000


# non % + priors

In [158]:
groupes = {
    'unknow': ['Euphysa spp.','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [159]:
priors = {
    'unknow': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 5},
    'Evadne': {'sigma': 5},
    'Verruca': {'sigma':1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma':1},
    'Beroe': {'mu': 7.12, 'sigma': 10},
    'Acartia': {'mu': 0.003, 'sigma': 0.1},
    'Metridia': {'mu': 0.08, 'sigma': 0.1},
    'Paracalanus': {'mu': 0.0016, 'sigma': 0.1},
    'Temora': {'mu': 0.001, 'sigma': 0.01},
    'Oithona': {'mu': 0.0005, 'sigma': 0.01},
    'Spinocalanus': {'mu': 0.2, 'sigma': 1},
    'Centropages': {'mu': 0.0075, 'sigma': 0.01},
    'Sagitta': {'mu': 0.6, 'sigma': 1}, #Chaetognatha
    'Pseudocalanus': {'mu': 0.004, 'sigma': 0.1},
    'Thysanoessa': {'mu': 6, 'sigma': 6}, #Euphosiacea
    'Oikopleura': {'mu': 0.006, 'sigma': 0.01},
    'Microcalanus': {'mu': 0.0002, 'sigma': 0.01},
    'Microsetella spp.': {'mu': 0.0008, 'sigma': 0.01},
    'Calanus finmarchicus': {'mu': 0.2, 'sigma': 1},
    'Calanus glacialis': {'mu': 0.2, 'sigma': 1},
    'Calanus hyperboreus': {'mu': 1.7, 'sigma': 2},
    'Bradyidius': {'mu': 0.3, 'sigma': 1},
    'Semibalanus': {'sigma': 5},
    'Chaetognatha': {'mu': 1, 'sigma': 1},
    'Euchaeta':  {'mu': 0.14, 'sigma': 1},
    'Eukrohnia': {'mu': 1.4, 'sigma': 1.4},
    'Hyperiidea':{'mu': 20, 'sigma': 20},
    'Themisto': {'mu': 1.4, 'sigma': 1.4},
    'Bolinopsis': {'mu': 40, 'sigma': 10},
    'Monstrilla': {'mu': 0.1, 'sigma': 1},
    'Oncaea': {'mu': 0.001, 'sigma': 0.1},
    'Pagurus':{'sigma': 5},
    'Paraeuchaeta': {'mu': 1, 'sigma': 1},
    'Pleurobrachia': {'mu': 2, 'sigma': 2},
    'Podon': {'sigma': 1},
    'Polychaeta':{'mu': 0.05, 'sigma': 0.05},
    'Thalestris':{'mu': 0.001, 'sigma': 0.01},
    'Tisbe':{'mu': 0.001, 'sigma': 0.01},
    'Undinella':{'mu': 0.3, 'sigma': 1},
    'Augaptilus':{'mu': 0.3, 'sigma': 1}, 
    'Obelia':{'mu': 1, 'sigma': 1},
    'Plotocnide':{'mu': 1, 'sigma': 1}, 
    'Scaphocalanus':{'mu': 0.2, 'sigma': 1},
    'Meganyctiphanes':{'mu': 10, 'sigma': 10}, 
    'Harpacticus':{'mu': 0.001, 'sigma': 0.01},
    'Calanus spp.':{'mu': 0.7, 'sigma': 0.7}, 
    'Gammaridea':{'mu': 3, 'sigma': 3}, 
    'Hormiphora cucumis':{'mu': 10, 'sigma': 10},
    'Jaschnovia tolli':{'mu': 3, 'sigma': 3}, 
    'Scolecithricella minor':{'mu': 0.1, 'sigma': 0.1}, 
    'Spionidae spp.':{'sigma': 1}, 
    'Temorites brevis':{'mu': 0.3, 'sigma': 1},
    'Aetideopsis multiserrata':{'mu': 0.3, 'sigma': 1},
    'Chiridius obtusifrons': {'mu': 0.3, 'sigma': 1},
    'Gaetanus brevispinus': {'mu': 0.2, 'sigma': 1}, 
    'Paraheterorhabdus compactus': {'mu': 0.3, 'sigma': 1},
    'Aglantha spp.':{'mu': 1, 'sigma': 1},
    'Candacia armata':{'mu': 0.1, 'sigma': 0.1},
    'Haloptilus acutifrons': {'mu': 0.1, 'sigma': 0.1},
    'Heterorhabdus norvegicus': {'mu': 1, 'sigma': 1},
    'Mormonilla minor': {'mu': 0.1, 'sigma': 0.1},
    'Rathkea': {'mu': 1, 'sigma': 1}, 
    'Sarsia': {'mu': 1, 'sigma': 1}, 
    'Alteutha interrupta': {'mu': 0.001, 'sigma': 0.001},
    'Cyclopina longicornis': {'mu': 0.001, 'sigma': 0.001},
    'Danielssenia typica': {'mu': 0.001, 'sigma': 0.001},
    'Zaus': {'mu': 0.001, 'sigma': 0.001}, 
    'Laophonte perplexa':{'mu': 0.001, 'sigma': 0.001},
    'Lubbockia glacialis':{'mu': 0.001, 'sigma': 0.001},
    'Parathalestris': {'mu': 0.001, 'sigma': 0.001},
    'Rhynchothalestris helgolandica': {'mu': 0.001, 'sigma': 0.001},
    'Tegastes falcatus': {'mu': 0.001, 'sigma': 0.001},
    'Mimocalanus distinctocephalus': {'mu': 0.03, 'sigma': 0.1},
    'Pseudochirella spectabilis': {'mu': 0.03, 'sigma': 0.1},
    'Discoconchoecia elegans' : {'mu': 0.0015, 'sigma': 0.1},
    'Spinocalanus sp.' : {'mu': 0.2, 'sigma': 1}
    
    
    
    


    
    

}

In [160]:
import pandas as pd

# Créer une liste pour stocker les tuples (espèce, valeur)
data = []

for species, values in priors.items():
    # Sélectionner 'mu' s'il est présent, sinon 'sigma'
    value = values.get('mu', values['sigma'])
    data.append((species, value))

# Créer un DataFrame à partir de la liste des tuples
SpeciesCmass = pd.DataFrame(data, columns=['Species', 'Value']).set_index('Species')

# Affichage du DataFrame
SpeciesCmass


,Value
Species,
unknow,1.0000
Limacina,1.0000
Undinopsis,1.0000
Balanus,5.0000
Evadne,5.0000
...,...
Tegastes falcatus,0.0010
Mimocalanus distinctocephalus,0.0300
Pseudochirella spectabilis,0.0300


In [161]:
import pandas as pd


# Assurez-vous que les indices de SpeciesCmass correspondent aux colonnes de COPEPOD80_reduite
SpeciesCmass = SpeciesCmass.reindex(COPEPOD80_reduite.columns)

# Calculer la biomasse totale pour chaque échantillon
RealTotalCmass = (COPEPOD80_reduite * SpeciesCmass['Value']).sum(axis=1)


In [162]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Spinocalanus sp., Discoconchoecia elegans, Pseudochirella spectabilis, Mimocalanus distinctocephalus, Tegastes falcatus, Rhynchothalestris helgolandica, Parathalestris, Lubbockia glacialis, Laophonte perplexa, Zaus, Danielssenia typica, Cyclopina longicornis, Alteutha interrupta, Sarsia, Rathkea, Mormonilla minor, Heterorhabdus norvegicus, Haloptilus acutifrons, Candacia armata, Aglantha spp., Paraheterorhabdus compactus, Gaetanus brevispinus, Chiridius obtusifrons, Aetideopsis multiserrata, Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Se

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 320 seconds.


In [163]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df


Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,0.796,0.002,1.891
Limacina,0.920,0.370,1.417
Undinopsis,0.789,0.000,1.820
Balanus,4.756,3.850,5.626
Evadne,4.864,3.093,6.480
...,...,...,...
Tegastes falcatus,0.001,0.000,0.003
Mimocalanus distinctocephalus,0.093,0.000,0.211
Pseudochirella spectabilis,0.092,0.000,0.209


In [164]:
R1= results_df.copy()

# non% + Prior non informative

In [165]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [166]:
priors = {
    'unknow': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 1},
    'Evadne': {'sigma': 1},
    'Verruca': {'sigma': 1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma': 1},
    'Beroe': {'sigma': 1},
    'Acartia': {'sigma': 1},
    'Metridia': {'sigma': 1},
    'Paracalanus': {'sigma': 1},
    'Temora': {'sigma': 1},
    'Oithona': {'sigma': 1},
    'Spinocalanus': {'sigma': 1},
    'Centropages': {'sigma': 1},
    'Sagitta': {'sigma': 1},
    'Pseudocalanus': {'sigma': 1},
    'Thysanoessa': {'sigma': 1},
    'Oikopleura': {'sigma': 1},
    'Microcalanus': {'sigma': 1},
    'Microsetella spp.': {'sigma': 1},
    'Calanus finmarchicus': {'sigma': 1},
    'Calanus glacialis': {'sigma': 1},
    'Calanus hyperboreus': {'sigma': 1},
    'Bradyidius': {'sigma': 1},
    'Semibalanus': {'sigma': 1},
    'Chaetognatha': {'sigma': 1},
    'Euchaeta':  {'sigma': 1},
    'Eukrohnia': {'sigma': 1},
    'Hyperiidea':{'sigma': 1},
    'Themisto': {'sigma': 1},
    'Bolinopsis': {'sigma': 1},
    'Monstrilla': {'sigma': 1},
    'Oncaea': {'sigma': 1},
    'Pagurus':{'sigma': 1},
    'Paraeuchaeta': {'sigma': 1},
    'Pleurobrachia': {'sigma': 1},
    'Podon': {'sigma': 1},
    'Polychaeta':{'sigma': 1},
    'Thalestris':{'sigma': 1},
    'Tisbe':{'sigma': 1},
    'Undinella':{'sigma': 1},
    'Augaptilus':{'sigma': 1},
    'Obelia':{'sigma': 1},
    'Plotocnide':{'sigma': 1}, 
    'Scaphocalanus':{'sigma': 1},
    'Meganyctiphanes':{'sigma': 1},
    'Harpacticus':{'sigma': 1},
    'Calanus spp.':{'sigma': 1},
    'Gammaridea':{'sigma': 1},
    'Hormiphora cucumis':{'sigma': 1},
    'Jaschnovia tolli':{'sigma': 1},
    'Scolecithricella minor':{'sigma': 1},
    'Spionidae spp.':{'sigma': 1},
    'Temorites brevis':{'sigma': 1},
    'Aetideopsis multiserrata':{'sigma': 1},
    'Chiridius obtusifrons': {'sigma': 1},
    'Gaetanus brevispinus': {'sigma': 1}, 
    'Paraheterorhabdus compactus': {'sigma': 1},
    'Aglantha spp.':{'sigma': 1},
    'Candacia armata':{'sigma': 1},
    'Haloptilus acutifrons': {'sigma': 1},
    'Heterorhabdus norvegicus': {'sigma': 1},
    'Mormonilla minor': {'sigma': 1},
    'Rathkea': {'sigma': 1}, 
    'Sarsia':{'sigma': 1}, 
    'Alteutha interrupta': {'sigma': 1},
    'Cyclopina longicornis': {'sigma': 1},
    'Danielssenia typica': {'sigma': 1},
    'Zaus': {'sigma': 1}, 
    'Laophonte perplexa':{'sigma': 1},
    'Lubbockia glacialis':{'sigma': 1},
    'Parathalestris': {'sigma': 1},
    'Rhynchothalestris helgolandica': {'sigma': 1},
    'Tegastes falcatus': {'sigma': 1},
    'Mimocalanus distinctocephalus': {'sigma': 1},
    'Pseudochirella spectabilis': {'sigma': 1},
    'Discoconchoecia elegans' : {'sigma': 1},
    'Spinocalanus sp.' : {'sigma': 1}
    
    
    
    


    
    

}

In [167]:
# Is all species in COPEPOD80_reduite has a prior?
especes_dans_COPEPOD80_reduite = set(COPEPOD80_reduite.columns)
especes_absentes_dans_priors = especes_dans_COPEPOD80_reduite - set(priors.keys())
especes_absentes_dans_priors

set()

In [168]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Spinocalanus sp., Discoconchoecia elegans, Pseudochirella spectabilis, Mimocalanus distinctocephalus, Tegastes falcatus, Rhynchothalestris helgolandica, Parathalestris, Lubbockia glacialis, Laophonte perplexa, Zaus, Danielssenia typica, Cyclopina longicornis, Alteutha interrupta, Sarsia, Rathkea, Mormonilla minor, Heterorhabdus norvegicus, Haloptilus acutifrons, Candacia armata, Aglantha spp., Paraheterorhabdus compactus, Gaetanus brevispinus, Chiridius obtusifrons, Aetideopsis multiserrata, Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Se

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 373 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.


In [169]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne (non priors)': biomass_means.values,
    'HDI 3% (non priors)': hdi_3.values,
    'HDI 97% (non priors)': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne (non priors),HDI 3% (non priors),HDI 97% (non priors)
Espece,,,
unknow,0.784,0.000,1.916
Limacina,0.835,0.272,1.355
Undinopsis,1.130,0.001,2.362
Balanus,3.817,2.946,4.686
Evadne,2.576,1.223,3.843
...,...,...,...
Tegastes falcatus,0.806,0.001,1.924
Mimocalanus distinctocephalus,0.784,0.000,1.854
Pseudochirella spectabilis,0.786,0.000,1.851


In [170]:
R2=results_df.copy()

# 6 groups

In [171]:
groupes = {
    
    'unknow': ['Euphysa spp.','Spionidae spp.','Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione', 'Balanus balanus', 'Balanus crenatus', 'Balanus spp.','Evadne nordmanni', 'Evadne spp.','Podon spp.', 'Podon leuckartii','Verruca spp.', 'Pagurus spp.', 'Semibalanus balanoides', 'Polychaeta','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Other':['Hormiphora cucumis', 'Gammaridea', 'Discoconchoecia elegans','Parasagitta elegans', 'Sagitta spp.', 'Eukrohnia spp.', 'Eukrohnia hamata', 'Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.', 'Hyperia spp.', 'Hyperiidea', 'Beroe spp.', 'Bolinopsis spp.', 'Pleurobrachia spp.', 'Chaetognatha', 'Obelia geniculata',  'Obelia longissima', 'Obelia spp.', 'Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.', 'Themisto abyssorum', 'Themisto spp.', 'Thysanoessa inermis', 'Thysanoessa spp.', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Meganyctiphanes norvegica', 'Meganyctiphanes spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa', 'Plotocnide borealis', 'Plotocnide spp.', 'Rathkea octopunctata', 'Rathkea spp.'],
    '0.0001': ['Temora longicornis', 'Oithona similis', 'Oithona atlantica','Microsetella spp.', 'Laophonte perplexa', 'Cyclopina longicornis', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Tisbe furcata', 'Thalestris gibba','Tegastes falcatus',  'Temora spp.', 'Rhynchothalestris helgolandica', 'Parathalestris croni', 'Parathalestris harpactoides',  'Oithona spp.', 'Oithona frigida', 'Harpacticus littoralis', 'Harpacticus spp.', 'Danielssenia typica', 'Alteutha interrupta'],
    '0.001': ['Pseudocalanus elongatus', 'Microcalanus pusillus', 'Microcalanus pygmaeus', 'Acartia spp.', 'Acartia longiremis', 'Lubbockia glacialis', 'Pseudocalanus gracilis', 'Pseudocalanus spp.', 'Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.',  'Microcalanus spp.', 'Centropages hamatus', 'Centropages spp.', 'Centropages typicus', 'Acartia bifilosa',  'Aglantha digitale', 'Acartia', 'Paracalanus spp.', 'Aglantha spp.'],
    '0.1': [ 'Metridia lucens', 'Metridia longa', 'Calanus glacialis', 'Calanus finmarchicus', 'Mormonilla minor', 'Mimocalanus distinctocephalus', 'Copepoda','Spinocalanus sp.', 'Monstrilla spp.', 'Undinopsis spp.', 'Euchaeta spp.', 'Undinella oblonga', 'Temorites brevis', 'Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis', 'Scolecithricella minor', 'Scaphocalanus brevicornis', 'Scaphocalanus magnus', 'Pseudochirella spectabilis', 'Paraheterorhabdus compactus', 'Jaschnovia tolli', 'Heterorhabdus norvegicus', 'Haloptilus acutifrons', 'Gaetanus brevispinus', 'Chiridius obtusifrons', 'Candacia armata',   'Bradyidius similis', 'Augaptilus glacialis', 'Aetideopsis multiserrata',  'Metridia spp.', 'Calanus spp.'],
    '1': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica', 'Calanus hyperboreus']
    
    
    


}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [172]:
priors = {
    'unknow': {'sigma': 1},
    'Other':{'sigma': 1},
    '0.0001':{'mu': 0.0001, 'sigma':0.1},
    '0.001':{'mu': 0.001, 'sigma': 0.1 },
    '0.1':{'mu': 0.1, 'sigma': 1},
    '1':{'mu': 1, 'sigma': 1}
}

In [173]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [1, 0.1, 0.001, 0.0001, Other, unknow]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.


In [174]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,1.104,0.773,1.416
Other,4.930,4.766,5.089
0.0001,0.008,0.000,0.021
0.001,0.040,0.029,0.051
0.1,0.191,0.181,0.200
1,1.669,0.752,2.735


In [175]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Euphysa spp.,1.104,0.773,1.416
Spionidae spp.,1.104,0.773,1.416
Limacina retroversa,1.104,0.773,1.416
Limacina spp.,1.104,0.773,1.416
Limacina helicina,1.104,0.773,1.416
...,...,...,...
Metridia spp.,0.191,0.181,0.200
Calanus spp.,0.191,0.181,0.200
Paraeuchaeta glacialis,1.669,0.752,2.735


In [176]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (6 groups)', 'HDI 3% (6 groups)', 'HDI 97% (6 groups)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (6 groups)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (6 groups)': data_subset['HDI 3%'].mean(),
            'HDI 97% (6 groups)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (6 groups)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (6 groups)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (6 groups)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
grouped_results

,Biomasse moyenne (6 groups),HDI 3% (6 groups),HDI 97% (6 groups)
unknow,1.104,0.773,1.416
Acartia,0.040,0.029,0.051
Metridia,0.191,0.181,0.200
Microcalanus,0.040,0.029,0.051
Temora,0.008,0.000,0.021
...,...,...,...
Chiridius obtusifrons,0.191,0.181,0.200
Candacia armata,0.191,0.181,0.200
Aetideopsis multiserrata,0.191,0.181,0.200
Calanus spp.,0.191,0.181,0.200


In [177]:
R3=grouped_results.copy()

# 6 groups + dominant species

In [178]:
groupes = {
    
    'unknow': ['Euphysa spp.', 'Spionidae spp.','Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione', 'Balanus balanus', 'Balanus crenatus', 'Balanus spp.','Evadne nordmanni', 'Evadne spp.','Podon spp.', 'Podon leuckartii','Verruca spp.', 'Pagurus spp.', 'Semibalanus balanoides', 'Polychaeta','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Other':['Hormiphora cucumis', 'Gammaridea', 'Discoconchoecia elegans','Parasagitta elegans', 'Sagitta spp.', 'Eukrohnia spp.', 'Eukrohnia hamata', 'Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.', 'Hyperia spp.', 'Hyperiidea', 'Beroe spp.', 'Bolinopsis spp.', 'Pleurobrachia spp.', 'Chaetognatha', 'Obelia geniculata',  'Obelia longissima', 'Obelia spp.', 'Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.', 'Themisto abyssorum', 'Themisto spp.', 'Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Meganyctiphanes norvegica', 'Meganyctiphanes spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa', 'Plotocnide borealis', 'Plotocnide spp.', 'Rathkea octopunctata', 'Rathkea spp.'],
    '0.0001': ['Microsetella spp.', 'Laophonte perplexa', 'Cyclopina longicornis', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Tisbe furcata', 'Thalestris gibba','Tegastes falcatus',  'Temora spp.', 'Rhynchothalestris helgolandica', 'Parathalestris croni', 'Parathalestris harpactoides',  'Oithona spp.', 'Oithona frigida', 'Harpacticus littoralis', 'Harpacticus spp.', 'Danielssenia typica', 'Alteutha interrupta'],
    '0.001': ['Lubbockia glacialis', 'Pseudocalanus gracilis', 'Pseudocalanus spp.', 'Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.',  'Microcalanus spp.', 'Centropages hamatus', 'Centropages spp.', 'Centropages typicus', 'Acartia bifilosa',  'Aglantha digitale', 'Acartia', 'Paracalanus spp.', 'Aglantha spp.'],
    '0.1': ['Mormonilla minor', 'Mimocalanus distinctocephalus', 'Copepoda','Spinocalanus sp.', 'Monstrilla spp.', 'Undinopsis spp.', 'Euchaeta spp.', 'Undinella oblonga', 'Temorites brevis', 'Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis', 'Scolecithricella minor', 'Scaphocalanus brevicornis', 'Scaphocalanus magnus', 'Pseudochirella spectabilis', 'Paraheterorhabdus compactus', 'Jaschnovia tolli', 'Heterorhabdus norvegicus', 'Haloptilus acutifrons', 'Gaetanus brevispinus', 'Chiridius obtusifrons', 'Candacia armata',   'Bradyidius similis', 'Augaptilus glacialis', 'Aetideopsis multiserrata',  'Metridia spp.', 'Calanus spp.'],
    '1': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Calanus fimarchicus': ['Calanus finmarchicus'],
    'Calanus glacialis': ['Calanus glacialis'],
    'Calanus hyperboreus': [ 'Calanus hyperboreus'],
    'Oithona atlantica': ['Oithona atlantica'],
    'Oithona similis': ['Oithona similis'],
    'Metridia longa': ['Metridia longa'],
    'Metridia lucens': [ 'Metridia lucens'],
    'Acartia longiremis': ['Acartia longiremis'],
    'Acartia spp.': ['Acartia spp.'],
    'Microcalanus pusillus':['Microcalanus pusillus'],
    'Microcalanus pygmaeus':['Microcalanus pygmaeus'],
    'Pseudocalanus elongatus':['Pseudocalanus elongatus'],
    'Temora longicornis':['Temora longicornis'],
    'Thysanoessa spp.':['Thysanoessa spp.']
    
    
    


}

COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [179]:
priors = {
    'unknow': {'sigma': 1},
    'Other':{'sigma': 1},
    '0.0001':{'mu': 0.0001, 'sigma':0.1},
    '0.001':{'mu': 0.001, 'sigma': 0.1 },
    '0.1':{'mu': 0.1, 'sigma': 1},
    '1':{'mu': 1, 'sigma': 1},
    'Calanus fimarchicus': {'mu': 0.2, 'sigma':1 },
    'Calanus glacialis': {'mu': 0.2, 'sigma':1 },
    'Calanus hyperboreus': {'mu': 2, 'sigma':1 },
    'Oithona atlantica': {'mu': 0.0001, 'sigma':0.1 },
    'Oithona similis': {'mu': 0.0001, 'sigma':0.1},
    'Metridia longa': {'mu': 0.1, 'sigma':0.1},
    'Metridia lucens': {'mu': 0.1, 'sigma':0.1},
    'Acartia longiremis': {'mu': 0.001, 'sigma':0.1},
    'Acartia spp.': {'mu': 0.001, 'sigma':0.1},
    'Microcalanus pusillus':{'mu': 0.001, 'sigma':0.1},
    'Microcalanus pygmaeus':{'mu': 0.001, 'sigma':0.1},
    'Pseudocalanus elongatus':{'mu': 0.001, 'sigma':0.1},
    'Temora longicornis':{'mu': 0.0001, 'sigma':0.1},
    'Thysanoessa spp.':{'mu': 6, 'sigma':6},
}

In [180]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Thysanoessa spp., Temora longicornis, Pseudocalanus elongatus, Microcalanus pygmaeus, Microcalanus pusillus, Acartia spp., Acartia longiremis, Metridia lucens, Metridia longa, Oithona similis, Oithona atlantica, Calanus hyperboreus, Calanus glacialis, Calanus fimarchicus, 1, 0.1, 0.001, 0.0001, Other, unknow]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 100 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [181]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,1.314,0.964,1.642
Other,4.051,3.804,4.297
0.0001,0.087,0.000,0.200
0.001,0.078,0.000,0.178
0.1,0.143,0.000,0.334
1,1.272,0.001,2.625
Calanus fimarchicus,0.197,0.187,0.207
Calanus glacialis,0.480,0.000,0.956
Calanus hyperboreus,1.708,0.542,2.743


In [182]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Euphysa spp.,1.314,0.964,1.642
Spionidae spp.,1.314,0.964,1.642
Limacina retroversa,1.314,0.964,1.642
Limacina spp.,1.314,0.964,1.642
Limacina helicina,1.314,0.964,1.642
...,...,...,...
Microcalanus pusillus,0.027,0.000,0.072
Microcalanus pygmaeus,0.081,0.000,0.190
Pseudocalanus elongatus,0.039,0.027,0.051


In [183]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (group 5%)', 'HDI 3% (group 5%)', 'HDI 97% (group 5%)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (group 5%)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (group 5%)': data_subset['HDI 3%'].mean(),
            'HDI 97% (group 5%)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (group 5%)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (group 5%)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (group 5%)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
grouped_results

,Biomasse moyenne (group 5%),HDI 3% (group 5%),HDI 97% (group 5%)
unknow,1.3140,0.964,1.642000
Acartia,0.0804,0.000,0.182800
Metridia,0.1250,0.000,0.269000
Microcalanus,0.0620,0.000,0.146667
Temora,0.0805,0.000,0.188000
...,...,...,...
Aetideopsis multiserrata,0.1430,0.000,0.334000
Calanus spp.,0.1430,0.000,0.334000
Calanus finmarchicus,0.1970,0.187,0.207000
Calanus glacialis,0.4800,0.000,0.956000


In [184]:
# Example: renaming specific columns
grouped_results = grouped_results.rename(columns={
    'Biomasse moyenne (group 5%)': 'Biomasse moyenne (6group+)',
    'HDI 3% (group 5%)': 'HDI 3% (6group+)',
    'HDI 97% (group 5%)': 'HDI 97% (6group+)'
    # Add more columns as needed
})


In [185]:
R4= grouped_results.copy()

# Summary Table Result

In [186]:
table = pd.concat([R1, R2, R3, R4, SpeciesCmass], axis=1)

In [187]:
table

,Biomasse moyenne,HDI 3%,HDI 97%,Biomasse moyenne (non priors),HDI 3% (non priors),HDI 97% (non priors),Biomasse moyenne (6 groups),HDI 3% (6 groups),HDI 97% (6 groups),Biomasse moyenne (6group+),HDI 3% (6group+),HDI 97% (6group+),Value
unknow,0.796,0.002,1.891,0.784,0.000,1.916,1.104,0.773,1.416,1.314,0.964,1.642,1.0000
Limacina,0.920,0.370,1.417,0.835,0.272,1.355,1.104,0.773,1.416,1.314,0.964,1.642,1.0000
Undinopsis,0.789,0.000,1.820,1.130,0.001,2.362,0.191,0.181,0.200,0.143,0.000,0.334,1.0000
Balanus,4.756,3.850,5.626,3.817,2.946,4.686,1.104,0.773,1.416,1.314,0.964,1.642,5.0000
Evadne,4.864,3.093,6.480,2.576,1.223,3.843,1.104,0.773,1.416,1.314,0.964,1.642,5.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tegastes falcatus,0.001,0.000,0.003,0.806,0.001,1.924,0.008,0.000,0.021,0.087,0.000,0.200,0.0010
Mimocalanus distinctocephalus,0.093,0.000,0.211,0.784,0.000,1.854,0.191,0.181,0.200,0.143,0.000,0.334,0.0300
Pseudochirella spectabilis,0.092,0.000,0.209,0.786,0.000,1.851,0.191,0.181,0.200,0.143,0.000,0.334,0.0300
Discoconchoecia elegans,0.080,0.000,0.189,0.820,0.002,1.876,4.930,4.766,5.089,4.051,3.804,4.297,0.0015


In [188]:
import pandas as pd

somme_par_echantillon = COPEPOD80_reduite.sum(axis=1)

# Trouver la contribution maximale de chaque espèce
max_par_espece = COPEPOD80_reduite.max(axis=0)

# Initialiser un DataFrame pour stocker la contribution maximale en pourcentage
contribution_max_pourcentage = pd.DataFrame(index=max_par_espece.index)

# Calculer la contribution maximale en pourcentage pour chaque espèce
for espece in max_par_espece.index:
    # Trouver l'échantillon où l'espèce a sa contribution maximale
    echantillon_max = COPEPOD80_reduite[espece].idxmax()
    # Calculer la contribution en pourcentage
    contribution_max_pourcentage.at[espece, 'Contribution Max (%)'] = (COPEPOD80_reduite.at[echantillon_max, espece] / somme_par_echantillon[echantillon_max]) * 100


In [189]:
table = pd.concat([table, contribution_max_pourcentage], axis=1)

In [190]:
table

,Biomasse moyenne,HDI 3%,HDI 97%,Biomasse moyenne (non priors),HDI 3% (non priors),HDI 97% (non priors),Biomasse moyenne (6 groups),HDI 3% (6 groups),HDI 97% (6 groups),Biomasse moyenne (6group+),HDI 3% (6group+),HDI 97% (6group+),Value,Contribution Max (%)
unknow,0.796,0.002,1.891,0.784,0.000,1.916,1.104,0.773,1.416,1.314,0.964,1.642,1.0,25.128821
Limacina,0.920,0.370,1.417,0.835,0.272,1.355,1.104,0.773,1.416,1.314,0.964,1.642,1.0,NaN
Undinopsis,0.789,0.000,1.820,1.130,0.001,2.362,0.191,0.181,0.200,0.143,0.000,0.334,1.0,NaN
Balanus,4.756,3.850,5.626,3.817,2.946,4.686,1.104,0.773,1.416,1.314,0.964,1.642,5.0,NaN
Evadne,4.864,3.093,6.480,2.576,1.223,3.843,1.104,0.773,1.416,1.314,0.964,1.642,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Microcalanus pusillus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.197190
Microcalanus pygmaeus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.614413
Pseudocalanus elongatus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.656909
Temora longicornis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.618045


In [142]:
table['Différence Biomasse moyenne (%)'] = ((table['Biomasse moyenne'] - table['Value']) / table['Biomasse moyenne']) * 100
table['Différence Biomasse moyenne (non priors) (%)'] = ((table['Biomasse moyenne (non priors)'] - table['Value']) / table['Biomasse moyenne (non priors)']) * 100
table['Différence Biomasse moyenne (6 groups) (%)'] = ((table['Biomasse moyenne (6 groups)'] - table['Value']) / table['Biomasse moyenne (6 groups)']) * 100
table['Différence Biomasse moyenne (6group+) (%)'] = ((table['Biomasse moyenne (6group+)'] - table['Value']) / table['Biomasse moyenne (6group+)']) * 100

# Organiser les lignes dans l'ordre croissant de 'Contribution Max (%)'
table = table.sort_values(by='Contribution Max (%)', ascending=True)

# Sauvegarder le DataFrame en fichier CSV
table.to_csv(Data_PATH + 'IM_sensitivity.csv', index=True)


table

,Biomasse moyenne,HDI 3%,HDI 97%,Biomasse moyenne (non priors),HDI 3% (non priors),HDI 97% (non priors),Biomasse moyenne (6 groups),HDI 3% (6 groups),HDI 97% (6 groups),Biomasse moyenne (6group+),HDI 3% (6group+),HDI 97% (6group+),Value,Contribution Max (%),Différence Biomasse moyenne (%),Différence Biomasse moyenne (non priors) (%),Différence Biomasse moyenne (6 groups) (%),Différence Biomasse moyenne (6group+) (%)
Sarsia,1.283,0.003,2.647,0.800,0.001,1.842,4.927,4.771,5.087,4.052000,3.817000,4.306000,1.0000,0.027496,22.057677,-25.000000,79.703674,75.320829
Alteutha interrupta,0.001,0.000,0.003,0.803,0.001,1.931,0.008,0.000,0.021,0.086000,0.000000,0.197000,0.0010,0.028562,0.000000,99.875467,87.500000,98.837209
Tegastes falcatus,0.001,0.000,0.003,0.818,0.000,1.886,0.008,0.000,0.021,0.086000,0.000000,0.197000,0.0010,0.047544,0.000000,99.877751,87.500000,98.837209
Aglantha spp.,1.276,0.001,2.627,0.799,0.002,1.966,0.040,0.028,0.051,0.079000,0.000000,0.190000,1.0000,0.070771,21.630094,-25.156446,-2400.000000,-1165.822785
Paraeuchaeta,1.288,0.002,2.609,0.792,0.001,1.855,1.637,0.618,2.612,1.265000,0.001000,2.618000,1.0000,0.090232,22.360248,-26.262626,38.912645,20.948617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Oithona,0.008,0.000,0.017,0.027,0.000,0.058,0.008,0.000,0.021,0.056000,0.000000,0.130250,0.0005,51.053856,93.750000,98.148148,93.750000,99.107143
Acartia,0.072,0.000,0.170,0.160,0.001,0.380,0.040,0.028,0.051,0.081600,0.000000,0.189400,0.0030,70.729312,95.833333,98.125000,92.500000,96.323529
Microcalanus,0.008,0.000,0.019,0.036,0.000,0.094,0.040,0.028,0.051,0.061667,0.000000,0.147000,0.0002,78.197190,97.500000,99.444444,99.500000,99.675676
Pseudocalanus,0.009,0.000,0.019,0.017,0.002,0.030,0.040,0.028,0.051,0.065667,0.008667,0.143333,0.0040,95.656909,55.555556,76.470588,90.000000,93.908629
